In [1]:
import numpy as np
from scipy.fft import dct, idct, dst, idst
from scipy.sparse import identity, kron, spdiags
from scipy.special import erf
import matplotlib.pyplot as plt
from icecream  import ic
from dct_dst import get_grid
from fft_tdse.subspace_imag_prop import ImagProp
from time import time

class ErfgauPotential:
    """Erfgau potential
    
    $$ V(r) = erf(mu*r)/r + c*exp(-alpha^2*r^2) $$
    
    Args:
    - mu: float, parameter in the potential


    
    """
    def __init__(self, mu=1.0):

        c = 0.923+1.568*mu
        alpha = 0.2411+1.405*mu

        self.mu = mu
        self.c = c
        self.alpha = alpha
        
        self.__call__ = self.potential

    def long_range(self, r):
        
        if r == 0:
            V0 = self.mu
        else:
            V0 = erf(self.mu*r)/r
            
        return V0


        
    def potential(self, x, y, z):

        r2 = (x*x+y*y+z*z)

        V0 = np.vectorize(self.long_range)(r2**.5)
            
        V = V0 + self.c*np.exp(-self.alpha**2*r2)
        return -V

    def potential_radial(self, r):

        r2 = r*r

        V0 = np.vectorize(self.long_range)(r)
            
        V = V0 + self.c*np.exp(-self.alpha**2*r2)
        return -V


In [2]:
# def dct4grid(a, b, n):
#     x = np.linspace(a, b, n+1)[1:]
#     h = x[1] - x[0]
#     x = x-h/2
    
#     f = np.arange(.5,n+.5) * np.pi / (b-a)
    
#     return x, f
    
class SinDvr:
    def __init__(self, a, b, n):
        self.a = a
        self.b = b
        self.n = n
        self.x_full, self.x, self.f = get_grid(a, b, n, kind='dst', type=2)
        # self.x = np.linspace(a, b, n+2)[1:-1]
        # self.f = np.arange(1,n+1) * np.pi / (b-a)

    def laplace(self, u):
        """ Returns the second derivative of u. """
        return -idst(self.f**2*dst(u, type=1), type=1)

    def exp_laplace(self, u, a):
        """ Compute exp(a*L) acting on u. """
        return idst(np.diag(np.exp(-a*self.f**2)) @ dst(u, type=1), type=1)

    def get_laplacian_matrix(self):
        """ Returns the Laplacian operator on the grid. """
        A_dense = np.zeros((self.n,self.n))
        for i in range(self.n):
            u = np.zeros(self.n)
            u[i] = 1
            A_dense[:,i] = self.laplace(u)  
        return A_dense   
        
class SinCosRadialDvr:
    def __init__(self, a, b, n):
        self.a = a
        self.b = b
        self.n = n
        self.x_full, self.x, self.f = get_grid(a, b, n, kind='dct', type=4) #dct4grid(a, b, n)
        
        
    def dctdiff(self, u):
        """ Differentiates a DCT-IV type grid function, and
        returns a DST-IV type grid function. """
        dudx =  idst(-self.f*dct(u, type=4), type=4)
        return dudx

    def dstdiff(self, u):
        """ Differentiates a DST-IV type grid function, and
        returns a DCT-IV type grid function. """
        dudx =  idct(self.f*dst(u, type=4), type=4)
        return dudx

    def laplace(self, u): # starts with a DCT-IV type grid function
        dudx =  self.dctdiff(u) # this is a DST-IV type grid function
        d2udx2 = self.dstdiff(dudx) # this is a DCT-IV type grid function
        return dudx / self.x + d2udx2 # this is a DCT-IV type grid function


    def get_laplacian_matrix(self):
        """ Returns the Laplacian operator on the grid. """
        A_dense = np.zeros((self.n,self.n))
        for i in range(self.n):
            u = np.zeros(self.n)
            u[i] = 1
            A_dense[:,i] = self.laplace(u)  
        return A_dense        



r_max = 30
n_r = 100
grid = SinCosRadialDvr(0, r_max, n_r)
T = -.5 * grid.get_laplacian_matrix()
#V = np.diag(erfgau.potential_radial(grid.x))
V = np.diag(0.5*grid.x**2)
H = T + V
E, U = np.linalg.eig(H)
idx = E.argsort()
E = E[idx]
U = U[:,idx]
ic(E)


ic| E: array([  1.        ,   3.        ,   5.        ,   7.        ,
                9.        ,  11.        ,  13.        ,  15.        ,
               17.        ,  19.        ,  21.        ,  23.        ,
               25.        ,  27.        ,  28.99999997,  30.99999982,
               32.99999902,  34.99999516,  36.99997785,  38.99990529,
               40.99962006,  42.99857334,  44.99506405,  46.98482917,
               48.96129132,  50.92717974,  52.92065777,  55.00930904,
               57.23685944,  59.60776958,  62.11125611,  64.73652425,
               67.47577389,  70.32364859,  73.27640074,  76.33131635,
               79.48636228,  82.73997075,  86.09090366,  89.53816416,
               93.08093669,  96.71854509, 100.45042249, 104.27608899,
              108.19513499, 112.2072083 , 116.31200411, 120.50925716,
              124.79873532, 129.18023455, 133.65357474, 138.21859622,
              142.87515698, 147.62313028, 152.46240266, 157.39287223,
              162.41

array([  1.        ,   3.        ,   5.        ,   7.        ,
         9.        ,  11.        ,  13.        ,  15.        ,
        17.        ,  19.        ,  21.        ,  23.        ,
        25.        ,  27.        ,  28.99999997,  30.99999982,
        32.99999902,  34.99999516,  36.99997785,  38.99990529,
        40.99962006,  42.99857334,  44.99506405,  46.98482917,
        48.96129132,  50.92717974,  52.92065777,  55.00930904,
        57.23685944,  59.60776958,  62.11125611,  64.73652425,
        67.47577389,  70.32364859,  73.27640074,  76.33131635,
        79.48636228,  82.73997075,  86.09090366,  89.53816416,
        93.08093669,  96.71854509, 100.45042249, 104.27608899,
       108.19513499, 112.2072083 , 116.31200411, 120.50925716,
       124.79873532, 129.18023455, 133.65357474, 138.21859622,
       142.87515698, 147.62313028, 152.46240266, 157.39287223,
       162.4144473 , 167.52704512, 172.73059081, 178.02501654,
       183.41026068, 188.88626717, 194.45298492, 200.11

In [5]:
z_max = 320
r_max = 320
n_r = 500
n_z = 500
r_grid = SinCosRadialDvr(0, r_max, n_r)
z_grid = SinDvr(-z_max, z_max, n_z)
T_r = -.5 * r_grid.get_laplacian_matrix()
T_z = -.5 * z_grid.get_laplacian_matrix()
r = r_grid.x
z = z_grid.x
rr, zz = np.meshgrid(r, z, indexing='ij')
V = ErfgauPotential(.5).potential(rr, 0, zz)
#H = np.kron(T_r, np.eye(n_z)) + np.kron(np.eye(n_r), T_z) + np.diag(V.flatten())
H_sparse = kron(T_r, identity(n_z)) + kron(identity(n_r), T_z) + spdiags(V.flatten(), 0, n_r*n_z, n_r*n_z, format='csr')
#ic(H.shape)
ic(H_sparse.shape, H_sparse.nnz)
# E, U = np.linalg.eig(H)
# idx = E.argsort()
# E = E[idx]
# U = U[:,idx]
# ic(E[:10])


ic| H_sparse.shape: (250000, 250000), H_sparse.nnz: 249750000


((250000, 250000), 249750000)

In [6]:
from scipy.linalg import expm

dt = 0.01
U_r = expm(-T_r*dt)
U_z = expm(-T_z*dt)
def step(psi):
    psi = np.exp(-.5*dt*V) * psi.reshape(n_r, n_z)
    psi = np.dot(U_r, psi)
    psi = np.dot(U_z, psi.T).T
    #psi = z_grid.exp_laplace(psi.T, dt/2).T
    #ic(psi.shape)
    psi = np.exp(-0.5*dt*V) * psi
    return psi.flatten()
        

n_vec = 3
ip = ImagProp(n_r*n_z, step)
#psi = np.random.rand(n_r*n_z, n_vec)
#psi[:,0] = np.exp(-(rr**2 + zz**2)**.5).flatten()
tic = time()
psi, error = ip.imag_prop(psi, maxit=2000, tol=1e-6)
toc = time()
ic(toc-tic)

K = psi.conj().T @ H_sparse @ psi
E, U = np.linalg.eigh(K)
ic(E)


NameError: name 'psi' is not defined

In [ ]:
from scipy.sparse.linalg import eigs
E_sparse, U_sparse = eigs(H_sparse, k=1, sigma=-.5)
idx = E_sparse.real.argsort()
E_sparse = E_sparse[idx]
U_sparse = U_sparse[:,idx]
ic(E_sparse[:10])

ic| E_sparse[:10]: array([-0.49610786+0.j])


array([-0.49610786+0.j])

In [7]:
E[:10]

array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19.])

In [26]:
help(dst)

Help on _Function in module scipy.fft._realtransforms:

dst(x, type=2, n=None, axis=-1, norm=None, overwrite_x=False, workers=None, orthogonalize=None)
    Return the Discrete Sine Transform of arbitrary type sequence x.
    
    Parameters
    ----------
    x : array_like
        The input array.
    type : {1, 2, 3, 4}, optional
        Type of the DST (see Notes). Default type is 2.
    n : int, optional
        Length of the transform. If ``n < x.shape[axis]``, `x` is
        truncated.  If ``n > x.shape[axis]``, `x` is zero-padded. The
        default results in ``n = x.shape[axis]``.
    axis : int, optional
        Axis along which the dst is computed; the default is over the
        last axis (i.e., ``axis=-1``).
    norm : {"backward", "ortho", "forward"}, optional
        Normalization mode (see Notes). Default is "backward".
    overwrite_x : bool, optional
        If True, the contents of `x` can be destroyed; the default is False.
    workers : int, optional
        Maxim